# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
from webdriver_manager.chrome import ChromeDriverManager
################ TIME MODLULES ###################
import time
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re

# Scrape [Google Finance](https://www.google.com/finance)

In [4]:
def scrape(company):
    # Initialize lists
    ticker=[]
    listed=[]
    public=[]
    # open chrome driver
    #driver=webdriver.Chrome(executable_path="C:\\Users\\Amit\\Desktop\\Internships\\BiosectRx\\Task_1\\chromedriver.exe")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    for c,i in enumerate(company):
        print(c,i)
        try:
            driver.get('https://www.google.com/search?tbm=fin&q=NASDAQ%3A%20{}'.format("+".join(i.split()))) # search company as NASDAQ listed
            source = driver.page_source
            soup=bs4.BeautifulSoup(source, 'html.parser')
            ticker.append(soup.find('span',{'jsname':'qRSVye'}).text[0]) # get ticker (up or down)
            listed.append(soup.find('div',{'class':'wx62f PZPZlf'}).text.split(':')[0]) # where is the company listed?
            public.append(1) # company is public if on NASDAQ
        except:
            time.sleep(5)
            try:
                driver.get('https://www.google.com/search?tbm=fin&q={}'.format("+".join(i.split()))) # search company without NASDAQ
                source = driver.page_source
                soup=bs4.BeautifulSoup(source, 'html.parser')
                ticker.append(soup.find('span',{'jsname':'qRSVye'}).text[0]) # get ticker (up or down)
                listed.append(soup.find('div',{'class':'wx62f PZPZlf'}).text.split(':')[0]) # where is the company listed?
                public.append(1) # company is public if on NYSE or SHA etc.
            except:
                # if company is not on NASDAQ, NYSE, SHA etc., consider it non-public
                ticker.append(np.nan) 
                listed.append(np.nan)
                public.append(0)
        time.sleep(5) # wait for 5 sec after every loop
    driver.close() # close driver
    return public,listed,ticker

In [ ]:
company=list(pd.read_excel('companies.xlsx')[' Life Science Company']) # read list of companies
public,listed,ticker=scrape(company) # scrape google finance
df=pd.DataFrame(zip(company,public,listed,ticker),columns=['Company','Public','Listed On','Ticker']) # create dataframe
company2=df.loc[(df['Public']==1)]['Company']
public,listed,ticker=scrape(company2) # update dataframe
df2=pd.DataFrame(zip(company2,public,listed,ticker),columns=['Company','Public','Listed On','Ticker'])
df2.index=list(df.loc[(df['Public']==1)].index)
df.update(df2)
df['Public']=df['Public'].astype(int)

In [15]:
df1=df.loc[(df['Public']==0)] # filter out non-public companies
df2=df.loc[(df['Public']==1)] # filter out public companies
df.shape,df1.shape

((451, 4), (432, 4))

In [16]:
# Only 19/451 companies are public, rest 432 are non-public
# public companies are the following
df2

,Company,Public,Listed On,Ticker
26,PB&B SA,1,NYSE,+
130,Akouos,1,NASDAQ,+
135,HitGen,1,SHA,+
147,Cyxone AB,1,FRA,−
175,MicroOpt,1,NASDAQ,−
200,HOOKIPA Pharma Inc.,1,NASDAQ,−
219,Viela Bio,1,NASDAQ,−
221,Viela Bio Inc.,1,NASDAQ,−
227,Autolus Therapeutics,1,NASDAQ,−
228,"TELA Bio, Inc.",1,NASDAQ,+


In [ ]:
# save to excel sheet
df1.to_excel('non-public.xlsx',index=False)
df2.to_excel('public.xlsx',index=False)